# Prototype

#### Data processing

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.stats import rankdata

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, GlobalMaxPool1D, Dropout
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model

In [2]:
reddit_df = pd.read_csv('../data/reddit_dataset.csv')
reddit_df.head()

,Unnamed: 0,id,comment,score
0,0,1,Dumbass liberals. You’d let anyone fuck you o...,0.979
1,1,2,"Yeah ok, shut the fuck up bitch. It’s Bernie o...",0.958
2,2,3,"fuck you, you're just pretending to be blocked...",0.938
3,3,4,"""He's a borderline rapist and that's just what...",0.938
4,4,5,"If you stick your dick in crazy, the crazy sti...",0.938


In [3]:
len(reddit_df)

6000

In [4]:
reddit_df.columns

Index(['Unnamed: 0', 'id', 'comment', 'score'], dtype='object')

In [5]:
#drop unnamed:0 colunm 

In [6]:
reddit_df.drop(['Unnamed: 0'], axis=1, inplace=True)

Drop rows with no-longer availble (deleted or removed) comments.

In [7]:
for index, row in reddit_df.iterrows():
    if row['comment'] == '[removed]' or row['comment'] == '[deleted]' or row['comment'] == 'this comment is no longer availble':
        reddit_df.drop(index, axis=0, inplace=True)

Checking if I miss any deleted or removed comment

In [8]:
for index, row in reddit_df.iterrows():
    if row['comment'] == '[removed]' or row['comment'] == '[deleted]':
        print('you missed something')

It confirmed that all rows with no comments are removed. 

In [9]:
len(reddit_df)

5721

In total of 245 rowsa are deleted. It reduce the number of data but it is something that I cannot do anything about it. 

Now, aplit dateframe into two dataframe : Train, test, and validation data.

In [10]:
#reset indexing of datafram 
reddit_df.reset_index(drop=True, inplace=True)

### Text cleaning 

Text cleaning process involves removing unncessary words such as stopwords, and remove punctuations. 

In [11]:
import string 
from nltk.corpus import stopwords

In [12]:
# def clean_text(text):
#     #tokenize text 
#     tokens = text.split()
#     #remove punctuation 
#     table = str.maketrans('', '', string.punctuation)
#     tokens = [w.translate(table) for w in tokens]
#     # remove remaining tokens that are not alphabetic
#     tokens = [word for word in tokens if word.isalpha()]
#     # filter out stop words
#     stop_words = set(stopwords.words('english'))
#     tokens = [w for w in tokens if not w in stop_words]
#     # filter out short tokens
#     tokens = [word for word in tokens if len(word) > 1]
#     return str(tokens)

In [13]:
# t = "I am saying, you cant be too generous as in, you shouldn't be too generous..."
# c = clean_text(t)
# print(c)

itterate through each text in dataframe and clean each text

In [14]:
# for index, row in reddit_df.iterrows():
#     reddit_df.iloc[index, 1] = clean_text(row['comment'])


In [16]:
# def translate(value, leftMin, leftMax, rightMin, rightMax):
#     # Figure out how 'wide' each range is
#     leftSpan = leftMax - leftMin
#     rightSpan = rightMax - rightMin

#     # Convert the left range into a 0-1 range (float)
#     valueScaled = float(value - leftMin) / float(leftSpan)

#     # Convert the 0-1 range into a value in the right range.
#     return rightMin + (valueScaled * rightSpan)

In [26]:
for index, row in reddit_df.iterrows():
    print(row['score'])
#     reddit_df.iloc[index, 2] = reddit_df['score'][index]

-0.354
-0.479
-0.38299999999999995
-0.125
-0.25
-0.45799999999999996
-0.16699999999999998
-0.149
0.33299999999999996
0.41700000000000004
-0.43799999999999994
-0.10400000000000001
-0.021
-0.125
-0.042
0.042
-0.20800000000000002
-0.021
0.146
0.304
-0.354
-0.292
-0.146
-0.043
-0.042
0.542
-0.10400000000000001
0.083
-0.16699999999999998
0.33299999999999996
-0.625
0.25
-0.312
-0.354
0.7709999999999999
0.22899999999999998
-0.5529999999999999
0.213
0.271
-0.042
0.511
0.0
-0.043
-0.10400000000000001
0.848
-0.20800000000000002
-0.25
0.125
-0.43799999999999994
-0.125
-0.125
-0.33299999999999996
0.10400000000000001
0.0
-0.312
-0.083
0.085
0.043
-0.354
0.021
-0.128
-0.10400000000000001
-0.021
0.021
-0.312
0.149
0.792
-0.22899999999999998
0.271
-0.083
0.083
0.467
0.083
-0.146
-0.255
-0.20800000000000002
-0.6809999999999999
0.833
-0.22899999999999998
-0.583
-0.312
-0.413
0.042
0.479
-0.312
-0.25
-0.085
-0.062
0.062
0.271
-0.22899999999999998
-0.292
-0.125
-0.125
0.312
-0.45799999999999996
-0.2080000

-0.174
-0.312
0.542
0.562
-0.22899999999999998
-0.062
0.043
0.354
-0.188
-0.20800000000000002
-0.10400000000000001
-0.062
-0.16699999999999998
-0.375
-0.20800000000000002
-0.354
-0.292
0.46799999999999997
-0.20800000000000002
-0.542
0.188
-0.021
-0.083
-0.083
-0.271
-0.292
0.213
-0.042
0.319
0.021
0.0
-0.25
0.292
0.062
0.188
0.042
-0.41700000000000004
-0.375
0.41700000000000004
-0.638
0.479
0.021
-0.021
-0.375
0.667
0.20800000000000002
0.7170000000000001
-0.083
-0.43799999999999994
-0.042
-0.083
0.16699999999999998
-0.25
0.042
-0.062
-0.43799999999999994
-0.083
-0.396
0.146
-0.45799999999999996
-0.239
-0.25
-0.062
-0.708
-0.489
-0.375
0.45799999999999996
-0.22899999999999998
-0.042
0.062
-0.188
0.6459999999999999
-0.25
-0.312
0.583
0.312
-0.083
0.125
-0.083
-0.021
-0.319
0.34
0.43799999999999994
-0.25
-0.213
0.604
0.062
0.292
0.542
0.125
-0.354
0.292
0.085
-0.562
0.0
-0.10400000000000001
-0.396
-0.085
-0.146
0.792
-0.146
-0.25
0.188
-0.33299999999999996
0.812
0.596
0.128
0.106
-0.271
-

0.479
0.479
-0.27699999999999997
-0.457
0.213
0.10400000000000001
0.021
0.435
-0.062
0.938
-0.6459999999999999
-0.021
0.16699999999999998
-0.146
-0.244
0.8540000000000001
-0.22899999999999998
-0.312
0.021
0.7609999999999999
0.271
-0.696
-0.583
0.255
-0.41700000000000004
0.521
-0.065
0.667
-0.10400000000000001
-0.41700000000000004
-0.36200000000000004
-0.583
-0.63
0.25
-0.191
0.542
0.188
-0.583
0.45799999999999996
0.596
0.062
-0.609
-0.271
-0.562
-0.41700000000000004
-0.532
-0.36200000000000004
-0.396
-0.625
-0.34
0.062
-0.25
-0.542
0.0
0.521
0.27699999999999997
-0.25
-0.255
-0.273
-0.271
0.0
-0.542
0.146
-0.16699999999999998
0.25
-0.10400000000000001
-0.25
-0.20800000000000002
-0.354
0.062
0.062
0.146
-0.43799999999999994
-0.292
-0.41700000000000004
0.37
0.271
0.083
-0.188
-0.25
-0.625
-0.33299999999999996
0.021
-0.062
-0.312
-0.292
-0.021
0.0
-0.604
-0.062
-0.312
-0.042
-0.062
-0.7659999999999999
-0.375
-0.312
0.708
0.375
-0.479
-0.16699999999999998
-0.37
-0.521
-0.021
-0.062
-0.292
0

-0.23399999999999999
0.083
0.10400000000000001
-0.188
0.875
-0.085
-0.196
0.106
0.45799999999999996
0.33299999999999996
-0.41700000000000004
-0.625
0.083
-0.146
0.396
-0.521
0.413
-0.021
-0.062
-0.22899999999999998
0.625
-0.271
-0.41700000000000004
-0.889
-0.125
0.708
-0.16699999999999998
0.042
0.20800000000000002
-0.188
0.28300000000000003
0.188
0.042
0.022000000000000002
-0.255
0.644
0.41700000000000004
0.33299999999999996
0.16699999999999998
0.16699999999999998
-0.20800000000000002
0.292
-0.375
0.312
-0.271
0.149
-0.312
0.083
-0.20800000000000002
0.292
-0.36200000000000004
-0.34
-0.16699999999999998
-0.562
-0.348
-0.271
0.062
-0.021
-0.042
0.13
-0.34
0.875
-0.292
0.43799999999999994
-0.128
-0.20800000000000002
0.25
-0.22899999999999998
-0.312
-0.2
-0.644
-0.10400000000000001
-0.083
0.271
0.146
-0.20800000000000002
-0.41700000000000004
-0.326
-0.41700000000000004
-0.562
-0.43799999999999994
0.133
-0.292
-0.188
-0.20800000000000002
-0.16699999999999998
0.45799999999999996
-0.146
0.319

-0.511
0.083
-0.20800000000000002
0.521
0.146
0.447
0.312
-0.213
0.083
0.021
0.062
-0.25
-0.604
-0.25
0.021
-0.062
-0.213
-0.087
-0.25
-0.271
-0.021
0.128
-0.16699999999999998
-0.213
0.20800000000000002
0.22899999999999998
-0.17
-0.191
0.125
0.25
-0.25
0.271
0.0
0.638
-0.27699999999999997
-0.128
-0.20800000000000002
0.042
-0.042
0.20800000000000002
-0.5
0.7390000000000001
0.45799999999999996
0.17800000000000002
-0.128
-0.45799999999999996
0.45799999999999996
0.625
0.20800000000000002
0.125
0.0
0.489
0.062
0.0
0.667
-0.125
-0.271
0.0
0.0
-0.217
0.5
-0.22899999999999998
-0.28300000000000003
0.43799999999999994
-0.396
-0.46799999999999997
0.062
0.083
0.354
0.452
0.604
-0.354
-0.062
-0.33299999999999996
-0.022000000000000002
0.16699999999999998
0.33299999999999996
0.312
0.8540000000000001
-0.354
-0.25
0.413
-0.354
-0.021
-0.062
-0.20800000000000002
-0.312
-0.149
0.20800000000000002
0.042
0.042
0.021
-0.688
-0.125
-0.083
-0.396
0.41700000000000004
-0.354
0.125
0.354
0.7020000000000001
0.021

In [18]:
#shuffle rows 
reddit_df = reddit_df.sample(frac=1).reset_index(drop=True)

In [19]:
reddit_df

,id,comment,score
0,5056,"Funny thing about the crab thing, I was sittin...",-0.354
1,5559,"Hello everyone, I'm in the market for some new...",-0.479
2,5223,Dangerous Angels by Francesca Lia Block,-0.383
3,3500,When i read the headline i told myself this wa...,-0.125
4,4445,A number of people have suggested boot media a...,-0.250
5,5524,This seems fair and should be priced into the ...,-0.458
6,3871,I believe NY state has a pending lawsuit with ...,-0.167
7,3710,inb4 four phantoms stunlock you off a cliff.,-0.149
8,860,When anybody (whether gay or straight) just.. ...,0.333
9,667,Does Farage still hold a position in your gove...,0.417


In [20]:
train_df = reddit_df.iloc[:4604,:]
val_df = reddit_df.iloc[4605:,:]

Train prototype model

In [31]:
MAX_SEQUENCE_LENGTH = 100
MAX_VOCAB_SIZE = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2
BATCH_SIZE = 128
EPOCHS = 10

In [51]:
# load in pre-tained word vectors
print("Loading word vectors...")
word2vec = {}
with open('./glove/glove.6B.100d.txt') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vec = np.asarray(values[1:], dtype='float32')
        word2vec[word] = vec
print('Found %s word vectors.' % len(word2vec))

Loading word vectors...
Found 400000 word vectors.


In [52]:
sentences = train_df['comment']
target = train_df['score']

In [53]:
# convert the sentences into sequences
tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE)
tokenizer.fit_on_texts(sentences)
sequences = tokenizer.texts_to_sequences(sentences)

In [54]:
# get word -> integer mapping
word2idx = tokenizer.word_index
# pad sequence so that we get N x T matrix
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
# prepare embedding matrix
print('Filling pre-trained embeddings...')
num_words = min(MAX_VOCAB_SIZE, len(word2idx) +1)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word2idx.items():
    if i < MAX_VOCAB_SIZE:
        embedding_vector = word2vec.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all zeros
            embedding_matrix[i] = embedding_vector

Filling pre-trained embeddings...


In [55]:
# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(
    num_words,
    EMBEDDING_DIM,
    weights=[embedding_matrix],
    input_length=MAX_SEQUENCE_LENGTH,
    trainable=False
)

In [56]:
print('Building model...')
input_ = Input(shape=(MAX_SEQUENCE_LENGTH,))
x = embedding_layer(input_)
x = Conv1D(128, 3, activation='relu')(x)
x = GlobalMaxPool1D()(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.1)(x)
output = Dense(1)(x)

Building model...


In [57]:
model = Model(input_, output)
model.compile(
    loss='mse',
    optimizer='adam'
)

In [58]:
r = model.fit(
    data,target,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_split=0.1,
    shuffle=True,
)

Epoch 1/10
33/33 [==============================] - 1s 33ms/step - loss: 0.2116 - val_loss: 0.0906
Epoch 2/10
33/33 [==============================] - 1s 35ms/step - loss: 0.0812 - val_loss: 0.0684
Epoch 3/10
33/33 [==============================] - 1s 33ms/step - loss: 0.0572 - val_loss: 0.0602
Epoch 4/10
33/33 [==============================] - 1s 31ms/step - loss: 0.0462 - val_loss: 0.0569
Epoch 5/10
33/33 [==============================] - 1s 31ms/step - loss: 0.0388 - val_loss: 0.0562
Epoch 6/10
33/33 [==============================] - 1s 25ms/step - loss: 0.0335 - val_loss: 0.0554
Epoch 7/10
33/33 [==============================] - 1s 25ms/step - loss: 0.0292 - val_loss: 0.0556
Epoch 8/10
33/33 [==============================] - 1s 25ms/step - loss: 0.0253 - val_loss: 0.0538
Epoch 9/10
33/33 [==============================] - 1s 32ms/step - loss: 0.0226 - val_loss: 0.0536
Epoch 10/10
33/33 [==============================] - 1s 26ms/step - loss: 0.0197 - val_loss: 0.0531


validating model

In [59]:
val_sequences = tokenizer.texts_to_sequences(val_df['comment'])
val_data = pad_sequences(val_sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [60]:
score = model.predict(val_data).flatten()

In [61]:
score[10:30]

array([-0.15999754, -0.2619876 ,  0.33375296,  0.02033173, -0.05411026,
        0.54923356, -0.64751476, -0.29404128,  0.11047223,  0.42854756,
        0.11045799,  0.12492898,  0.12552354, -0.06543443, -0.01900388,
       -0.27230147, -0.1424948 , -0.06385826, -0.17276989, -0.5850178 ],
      dtype=float32)

In [62]:
val_df['score'][10:30]

4615    0.438
4616   -0.229
4617    0.596
4618   -0.062
4619   -0.043
4620    0.708
4621   -0.688
4622    0.042
4623   -0.064
4624    0.708
4625    0.479
4626    0.106
4627    0.250
4628   -0.261
4629    0.229
4630   -0.312
4631   -0.191
4632   -0.083
4633    0.042
4634   -0.604
Name: score, dtype: float64